In [7]:
# Importing Dependencies

# Utility
import pandas as pd
import numpy as np

# Language Processing
import nltk
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
%pip install tweet-preprocessor
import preprocessor as p
nltk.download('stopwords')

# Word vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\suhan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\suhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df = pd.read_csv("Data/rawData.csv")

df.drop_duplicates('tweet_text',inplace = True, ignore_index = True)
df = df[df["cyberbullying_type"] != "other_cyberbullying"] #In order to remove ambiguity

df = df[df['tweet_text'].str.len() >= 10]
df = df[df['tweet_text'].str.len() <= 140]

df.describe()

,tweet_text,cyberbullying_type
count,25089,25089
unique,25089,5
top,"In other words #katandandre, your food was cra...",not_cyberbullying
freq,1,7819


In [9]:
# A set of useless words to remove
stopwords_list = stopwords.words('english')
additional_stopwords = ['rt', 'mkr', 'didn', 'bc', 'n', 'm', 'im', 'll', 'y', 've', 'u', 'ur', 'don', 't', 's']   # twitter lingo
stopwords_list.extend(additional_stopwords)

# A set of functions to help clean up the tweets.

def remove_punctuation(tweet):
    for mark in punctuation:
        tweet = tweet.replace(mark, '')
    return tweet

def tokenization(tweet):
    return word_tokenize(tweet)

lemmatizer = WordNetLemmatizer()
def lemmatize(tweet):
    return [lemmatizer.lemmatize(word) for word in tweet]

def remove_stopwords(tweet):
  res = []
  for word in tweet:
    if word not in stopwords_list:
      res.append(word)

  return res

def remove_stopwords(tweet):
    return [word for word in tweet if word not in stopwords_list]

def processing(tweet):
  tweet = p.clean(tweet)
  tweet = remove_punctuation(tweet)
  tweet = tweet.lower()
  tokens = tokenization(tweet)
  tokens = remove_stopwords(tokens)
  tokens = lemmatize(tokens)
  cleaned_tweet = ' '.join(tokens)
  return cleaned_tweet

df['cleaned_tweets'] = df['tweet_text'].apply(processing)
encoding = {'not_cyberbullying':0, 'gender':1, 'religion':2, 'age':3, 'ethnicity':4} # Makes classification easier
df['cyberbullying_type'] = df['cyberbullying_type'].map(encoding)
df.to_csv('Data/processedData.csv', index=False)


In [10]:
vectorizer = TfidfVectorizer(max_features=16)

tfidf_matrix = vectorizer.fit_transform(df['cleaned_tweets'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray())
tfidf_df['Labels'] = df['cyberbullying_type'].values

tfidf_df.drop(tfidf_df.columns[0], axis=1, inplace=True)

In [13]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tfidf_df = shuffle(tfidf_df)
tfidf_df.to_csv("Data\\finalData.csv",index=False)